In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.worksheet import Worksheet
import os
import datetime
import re
from tools_warehouse_scraper import get_toolswarehouse_price
from kennedys_scraper import get_kennedys_price
from vektools_scraper import get_vektools_price
from sydney_tools_scraper import get_sydney_tools_price
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


In [5]:
df = pd.read_excel('Pricing.xlsx',sheet_name="Sheet2")
df.columns

Index(['BRAND', 'PRODUCT SKU', 'PRODUCT NAME', 'Shop Name', 'PRODUCT LINK',
       'Note', 'BUNDLE LINK', 'Comment', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [7]:
df['PRODUCT SKU'] = df['PRODUCT SKU'].str.strip().str.upper()
df['PRODUCT NAME'] = df['PRODUCT NAME'].str.strip().str.upper()
df['Shop Name'] = df['Shop Name'].str.strip().str.upper()

# Append ' EBAY' to "Shop Name" where it equals 'WA INDUSTRIAL SUPPLIES'
df.loc[df['Shop Name'] == 'WA INDUSTRIAL SUPPLIES', 'Shop Name'] += ' EBAY'
df.loc[df['Shop Name'] == 'WElDERS ONLINE', 'Shop Name'] += ' EBAY'

df_sub = df[['BRAND', 'PRODUCT SKU', 'PRODUCT NAME', 'Shop Name', 'PRODUCT LINK',
       'Note', 'BUNDLE LINK', 'Comment']].copy()

In [9]:
# Forward fill only select columns
df_sub[['BRAND', 'PRODUCT SKU','PRODUCT NAME']] = df_sub[['BRAND', 'PRODUCT SKU','PRODUCT NAME']].ffill()


df_sub.head()

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment
0,Unimig,U11005K,VIPER 135,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...,NaN,NaN,NaN
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN
2,Unimig,U11005K,VIPER 135,HAMPDON EBAY,https://www.ebay.com.au/sch/i.html?_from=R40&_...,NaN,NaN,NaN
3,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES WEBSITE,https://www.waindustrialsupplies.net/product/v...,NaN,NaN,NaN
4,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186207493192?epid=...,NaN,NaN,NaN


In [11]:
#Example for Electroweld subset
elctro = df_sub[df_sub['Shop Name'].str.contains('electroweld', case=False, na=False, regex=True)][['Shop Name','PRODUCT LINK']]

elctro

,Shop Name,PRODUCT LINK
0,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
1,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...
22,ELECTROWELD EBAY,https://www.ebay.com.au/itm/285104945700?itmme...
25,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
35,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
...,...,...
1033,ELECTROWELD EBAY,NaN
1061,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
1062,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275214903410?_skw=...
1092,ELECTROWELD WEBSITE,NaN


In [13]:
import httpx
from parsel import Selector
import numpy as np
import pandas as pd

def get_ebay_price(url: str) -> str:
    """
    Given an eBay product URL, fetches the page and returns the original price.
    If a timeout occurs, returns np.nan.
    """
    # First check if url is a valid, non-empty string.
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan

    # Now safe to check length (though this is redundant with url.strip() check)
    if len(url) == 0:
        return -1
    
    # establish our HTTP2 client with browser-like headers
    session = httpx.Client(
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Language": "en-US,en;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
        },
        http2=True,
        follow_redirects=True
    )
    
    try:
        # Fetch the webpage with an explicit timeout of 15 seconds.
        response = session.get(url, timeout=15.0)
    except httpx.ReadTimeout:
        return np.nan
    except Exception as e:
        return np.nan

    sel = Selector(response.text)
    
    # helper function to extract and strip text via CSS selectors
    css = lambda query: sel.css(query).get(default="").strip()
    
    # extract the original price
    price_original = css(".x-price-primary>span::text")
    
    return price_original

# Example usage:
if __name__ == "__main__":
    url = "https://www.ebay.com.au/itm/275880137475?itmmeta=01J3RRGZ6RGRZ1PA0NB89D3ANK&hash=item403bbcd303:g:FG4AAOSwW1lmc0i2:sc:AU_RegularParcelWithTrackingAndSignature!2190!AU!-1&itmprp=enc%3AAQAJAAAA0Jo7zG6ZrYn%2F5GTELNdvHESu%2F8%2BPfiWY9kU--0LUo18ZDC%2BkIaIZvbRp1qI9So5TpISckKJtf4oGgV7V5XLYXMhG%2FLVX497kkF4F%2BcZyu0ELdaH4TYhi3PZ%2B%2BG7xxw75rLO07cwbga6GANQK6eH5xnlQBFG02TBJBi4CCfYH8LVydeSYVPiJ8rWQKbaKABcsxdxl%2B4uJjP6Ops8YFB%2BzE7T3l62OMSCzA63O1oEmgLiiOQySiIwlAiRx%2Fu3oj4L0GrlW2ZSPf1mHsjewL9BaDsQ%3D%7Ctkp%3ABk9SR7zzw5ieZA"
    print(get_ebay_price(url))


In [7]:
def get_electroweld_website_price(url: str) -> str:
    """
    Fetch the product page from the given URL and return the product price as a string.
    """
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return -1
    if not url.startswith("https://www.electroweld.com.au/product/"):
        return -1
    session = httpx.Client(
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                          "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        },
        follow_redirects=True
    )
    try:
        response = session.get(url)
    except httpx.ReadTimeout:
        return np.nan
    except Exception as e:
        return np.nan
    sel = Selector(response.text)
    css = lambda query: sel.css(query).get(default="").strip()
    

    # Extract the price text from the page.
    price_text = css("p.w-post-elm.product_field.price span.woocommerce-Price-amount.amount bdi::text")
    # Remove a leading currency symbol if present.
    if price_text.startswith("$"):
        price_text = price_text[1:]
    return price_text

url = "https://www.electroweld.com.au/product/unimig-viper-135-multi-3-in-1-mig-tig-stick-welder-welding-torch-mma-u11005k/"
price = get_electroweld_website_price(url)
print("Product Price:", price)

Product Price: 379.00


In [8]:
def get_bilba_website_price(url: str) -> str:

    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return np.nan
    if not url.startswith("https://bilba.com.au/products"):
        return np.nan
    """
    Fetch the product page from the given URL and return the product price as a string.
    The price is extracted from a <span> element with class "price-item price-item-regular".
    """
    # Create an HTTP client with appropriate headers.
    with httpx.Client(
        headers={
            "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                           "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"),
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        },
        follow_redirects=True
    ) as client:
        response = client.get(url)

    # Use parsel to parse the HTML content.
    sel = Selector(response.text)
    
    # Extract the price text from the <span> element.
    price_text = sel.css("span.price-item.price-item-regular::text").get(default="").strip()
    
    # Optionally remove any leading currency symbol, if needed.
    if price_text.startswith("$"):
        price_text = price_text[1:]
        
    return price_text
url = "https://bilba.com.au/products/unimig-razor-multi-175-welder?srsltid=AfmBOoqrcyItnYkdHvmmHMrp6Z7N-pFgOuiv_wJhyPmV19CKmUt5LIBi"
price = get_bilba_website_price(url)
print( price)

1,439.00


In [9]:
def get_gentronics_website_price(url: str) -> str:
    """
    Fetch the product page from the given URL and return the product price as a string.
    The price is stored in a <p> element with class="gentronics-price price".
    """
    # Basic checks for URL validity
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return np.nan
    
    # Check that the URL starts with the desired domain
    if not url.startswith("https://www.googleadservices.com/pagead/aclk") and not url.startswith("https://www.gentronics.com.au/"):
        return np.nan
    
    # Create an HTTP client with appropriate headers
    session = httpx.Client(
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                          "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        },
        follow_redirects=True
    )
    
    # Fetch the page
    try:
        response = session.get(url)
    except:
        return np.nan
    sel = Selector(response.text)
    
    # Helper function for CSS selection
    css = lambda query: sel.css(query).get(default="").strip()
    
    # Extract the price text from <p class="gentronics-price price">
    price_text = css("p.gentronics-price.price::text")
    
    # Remove a leading currency symbol if present
    if price_text.startswith("$"):
        price_text = price_text[1:]
    price_text = price_text.replace("per item", "").strip()
    
    return price_text

# Example usage
if __name__ == "__main__":
    test_url = (
        "https://www.googleadservices.com/pagead/aclk?sa=L&ai=DChcSEwjZ9uvXloaLAxU2qGYCHVOQD4oYABAEGgJzbQ&ae=2&aspm=1&co=1&ase=5&gclid=Cj0KCQiAhbi8BhDIARIsAJLOlufVl2KRd0vNv4v9hFa9sC_238MQii4XHsBkOTRn3QUpKMJwPGiQne4aAoLtEALw_wcB&ohost=www.google.com&cid=CAESVeD2k-VpUGbx3aGmilsMu45ZMV1Bprr20N9Gm98bkCSwJQ8jp06PNpCmx-boJX920KleNkC6xKJ8zDKPZs-bKKEYQsYqhnX8VLcD2NqlOMrbMcc2p-A&sig=AOD64_03qiZGD-PJa-lfD8uzdS3I3IM9kQ&ctype=5&q=&ved=2ahUKEwjTsubXloaLAxXByTgGHQiRMmYQww8oAnoECAgQCg&adurl="
        # your full URL here
    )
    price = get_gentronics_website_price(test_url)
    print(price)


1266.63


In [10]:
def get_weld_com_au_price(url: str) -> str:
    """
    Fetch the product page from the given URL and return the product price as a string.
    The price is stored in:
    <p class="price">
        <span class="woocommerce-Price-amount amount">
            <bdi>
                <span class="woocommerce-Price-currencySymbol">$</span>1,669.00
            </bdi>
        </span>
    </p>
    """
    # Basic checks for URL validity
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return -1
    # Ensure the URL starts with the Weld product URL base.
    if not url.startswith("https://www.weld.com.au/product/"):
        return -1
    
    # Create an HTTP client with appropriate headers.
    session = httpx.Client(
        headers={
            "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                           "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"),
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        },
        follow_redirects=True
    )
    
    # Fetch the page
    response = session.get(url)
    sel = Selector(response.text)
    css = lambda query: sel.css(query).get(default="").strip()
    
    # Extract the price text using the appropriate CSS selector.
    price_text = css("p.price span.woocommerce-Price-amount.amount bdi::text")
    
    # Remove a leading currency symbol if present.
    if price_text.startswith("$"):
        price_text = price_text[1:]
    
    return price_text

# Example usage:
if __name__ == "__main__":
    url = "https://www.weld.com.au/product/unimig-razor-multi-175-bundle-pk11091/?v=6502139931c4"
    price = get_weld_com_au_price(url)
    print( price)


1,669.00


In [11]:
def get_weldconnect_price(url: str) -> str:

    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return np.nan
    if not url.startswith("https://www.weldconnect.com.au/"):
        return np.nan

    try:
        with httpx.Client(
            headers={
                "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                               "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"),
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            },
            follow_redirects=True
        ) as session:
            # Set an explicit timeout (in seconds) for the request.
            response = session.get(url, timeout=15.0)
    except httpx.ReadTimeout:
        # If a timeout occurs, return np.nan (or any other appropriate fallback)
        return np.nan

    sel = Selector(response.text)
    # Extract the price using the "content" attribute from the <div> element.
    price_text = sel.css("div.h1[itemprop='price']::attr(content)").get(default="").strip()
    
    return price_text



In [12]:
import httpx
import numpy as np
import pandas as pd
from parsel import Selector

def get_metweld_price(url: str) -> str:
    """
    Fetch the product page from the given URL and return the product price as a string.
    The price is stored in an element like:
    
    <span data-product-price-with-tax="" class="price price--withTax">$979.00</span>
    
    Returns np.nan for any failure.
    """
    # Validate URL
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return np.nan
    if not url.startswith("https://metweld.com.au/"):
        return np.nan

    try:
        # Create an HTTP client with browser-like headers
        with httpx.Client(
            headers={
                "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                               "AppleWebKit/537.36 (KHTML, like Gecko) "
                               "Chrome/113.0.0.0 Safari/537.36"),
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            },
            follow_redirects=True
        ) as session:
            # Fetch the page with a 15-second timeout
            response = session.get(url, timeout=15.0)
    except (httpx.ReadTimeout, Exception):
        return np.nan

    sel = Selector(response.text)
    # Helper function for CSS selection
    css = lambda query: sel.css(query).get(default="").strip()
    
    # Extract the price text from the <span> element
    price_text = css("span.price.price--withTax::text")
    
    # Remove a leading currency symbol if present
    if price_text.startswith("$"):
        price_text = price_text[1:]
    
    return price_text

# Example usage:
if __name__ == "__main__":
    url = "https://metweld.com.au/viper-185-mig-tig-stick-welder/"
    price = get_metweld_price(url)
    print("Product Price:", price)


Product Price: 999.00


In [13]:
import httpx
import numpy as np
import pandas as pd
from parsel import Selector

def get_toolkitdepot_price(url: str) -> str:
    # Validate URL
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if not url.startswith("https://toolkitdepot.com.au/"):
        return np.nan

    try:
        # Create an HTTP client with browser-like headers.
        with httpx.Client(
            headers={
                "User-Agent": (
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/113.0.0.0 Safari/537.36"
                ),
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            },
            follow_redirects=True
        ) as session:
            # Fetch the page (timeout=15 seconds)
            response = session.get(url, timeout=15.0)
    except (httpx.ReadTimeout, Exception) as e:
        #print("Error fetching the page:", e)
        return np.nan

    # Print debug info about response length (optional)
    #print("DEBUG: Response length =", len(response.text), "characters")

    # Parse the HTML response using Parsel
    sel = Selector(response.text)

    # Try to extract the price by selecting the nested <span> inside the price container.
    price_text = sel.css("span.price.price--withTax span::text").get(default="").strip()
    
    # Fallback: if the above selector returns empty, concatenate all text nodes within the element.
    if not price_text:
        price_text = "".join(sel.css("span.price.price--withTax ::text").getall()).strip()

    # Remove a leading currency symbol if present
    if price_text.startswith("$"):
        price_text = price_text[1:]
    
    return price_text

if __name__ == "__main__":
    url = "https://toolkitdepot.com.au/unimig-viper-multi-135-mig-tig-stick-welder-u11005k/"
    price = get_toolkitdepot_price(url)
    print(price)

359.00


In [14]:
import httpx
import numpy as np
import pandas as pd
from parsel import Selector

def get_supercheapauto_price(url: str) -> str:
    """
    Fetch the product page from the given Supercheapauto URL and return the product price as a string.
    The price can be found in the HTML via XPath such as:
      Full absolute path: /html/body/div[1]/div[14]/div[3]/div/div[2]/div/div[3]/span/span/text()
      Or via classes: //span[contains(@class, 'price-sales')]//span[contains(@class, 'promo-price')]/text()
    Returns np.nan for any failure.
    """
    # URL validations
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if not url.startswith("https://www.supercheapauto.com.au/"):
        return np.nan

    # Create an HTTP client with browser-like headers
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/113.0.0.0 Safari/537.36"
        ),
        "Accept": (
            "text/html,application/xhtml+xml,application/xml;"
            "q=0.9,image/avif,image/webp,image/apng,*/*;"
            "q=0.8,application/signed-exchange;v=b3;q=0.7"
        ),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
    }

    try:
        with httpx.Client(headers=headers, follow_redirects=True) as session:
            response = session.get(url, timeout=15.0)
    except (httpx.ReadTimeout, Exception) as e:
        # Could also log the exception e if desired
        return np.nan

    # Parse the HTML using Selector
    sel = Selector(response.text)

    # First try to extract using class-based XPath
    price_class_nodes = sel.xpath("//span[contains(@class, 'price-sales')]//span[contains(@class, 'promo-price')]/text()").getall()
    price_class = "".join(price_class_nodes).strip()

    # Then try using the full absolute XPath (if class-based extraction is empty)
    price_full = sel.xpath("/html/body/div[1]/div[14]/div[3]/div/div[2]/div/div[3]/span/span/text()").get(default="").strip()

    # Choose the first non-empty result
    price_text = price_class if price_class else price_full

    # If the price text starts with a currency symbol, remove it
    if price_text.startswith("$"):
        price_text = price_text[1:].strip()

    # Return np.nan if no price was found
    if not price_text:
        return np.nan

    return price_text

# Example usage:
if __name__ == "__main__":
    url = "https://www.supercheapauto.com.au/p/unimig-unimig-viper-185-multiprocess-welder/678408.html"
    price = get_supercheapauto_price(url)
    print("Product Price:", price)


Product Price: 939.00


In [15]:
df_sub.head()

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment
0,Unimig,U11005K,VIPER 135,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...,NaN,NaN,NaN
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN
2,Unimig,U11005K,VIPER 135,HAMPDON EBAY,https://www.ebay.com.au/sch/i.html?_from=R40&_...,NaN,NaN,NaN
3,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES WEBSITE,https://www.waindustrialsupplies.net/product/v...,NaN,NaN,NaN
4,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186207493192?epid=...,NaN,NaN,NaN


In [16]:
df_biba_website = df_sub[
    df_sub['Shop Name'].str.contains('Bilba', case=False, na=False) &
    ~df_sub['Shop Name'].str.contains('BILBA EBAY', case=False, na=False)
]
df_biba_website['Price']= df_biba_website['PRODUCT LINK'].apply(get_bilba_website_price)
df_biba_website['Price_Bundle']= df_biba_website['BUNDLE LINK'].apply(get_bilba_website_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/2258500888.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_biba_website['Price']= df_biba_website['PRODUCT LINK'].apply(get_bilba_website_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/2258500888.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_biba_website['Price_Bundle']= df_biba_website['BUNDLE LINK'].apply(get_bilba_website_price)


In [17]:
df_electroweld_website = df_sub[df_sub['Shop Name'].str.contains('electroweld website', case=False, na=False, regex=True)]
df_electroweld_website['Price']= df_electroweld_website['PRODUCT LINK'].apply(get_electroweld_website_price)
df_electroweld_website['Price_Bundle']= df_electroweld_website['BUNDLE LINK'].apply(get_electroweld_website_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/3657741474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_electroweld_website['Price']= df_electroweld_website['PRODUCT LINK'].apply(get_electroweld_website_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/3657741474.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_electroweld_website['Price_Bundle']= df_electroweld_website['BUNDLE LINK'].apply(get_electroweld_website_price)


In [18]:
# Subset dataframe to retain only EBAY shop records
df_ebay = df_sub[df_sub['Shop Name'].str.contains('EBAY', case=False, na=False, regex=True)]
# Apply the get_ebay_price() on entire ebad df for fetching price for all applicable url rows
df_ebay['Price']= df_ebay['PRODUCT LINK'].apply(get_ebay_price)
df_ebay['Price_Bundle']= df_ebay['BUNDLE LINK'].apply(get_ebay_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1417790290.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ebay['Price']= df_ebay['PRODUCT LINK'].apply(get_ebay_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1417790290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ebay['Price_Bundle']= df_ebay['BUNDLE LINK'].apply(get_ebay_price)


In [19]:
df_gentronics = df_sub[df_sub['Shop Name'].str.contains('GENTRONICS', case=False, na=False, regex=True)]
df_gentronics['Price']= df_gentronics['PRODUCT LINK'].apply(get_gentronics_website_price)
df_gentronics['Price_Bundle']= df_gentronics['BUNDLE LINK'].apply(get_gentronics_website_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/2789824565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gentronics['Price']= df_gentronics['PRODUCT LINK'].apply(get_gentronics_website_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/2789824565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gentronics['Price_Bundle']= df_gentronics['BUNDLE LINK'].apply(get_gentronics_website_price)


In [20]:
df_weldconnect = df_sub[df_sub['Shop Name'].str.contains('WELDCONNECT', case=False, na=False, regex=True)]
df_weldconnect['Price']= df_weldconnect['PRODUCT LINK'].apply(get_weldconnect_price)
df_weldconnect['Price_Bundle']= df_weldconnect['BUNDLE LINK'].apply(get_weldconnect_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1885032458.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weldconnect['Price']= df_weldconnect['PRODUCT LINK'].apply(get_weldconnect_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1885032458.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weldconnect['Price_Bundle']= df_weldconnect['BUNDLE LINK'].apply(get_weldconnect_price)


In [21]:
df_weld_com = df_sub[df_sub['Shop Name'].str.contains('WELD.COM.AU', case=False, na=False, regex=True)]
df_weld_com['Price']= df_weld_com['PRODUCT LINK'].apply(get_weld_com_au_price)
df_weld_com['Price_Bundle']= df_weld_com['BUNDLE LINK'].apply(get_weld_com_au_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1079567426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weld_com['Price']= df_weld_com['PRODUCT LINK'].apply(get_weld_com_au_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1079567426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weld_com['Price_Bundle']= df_weld_com['BUNDLE LINK'].apply(get_weld_com_au_price)


In [22]:
df_metro_welder_service = df_sub[df_sub['Shop Name'].str.contains('METRO WELDER SERVICE', case=False, na=False, regex=True)]
df_metro_welder_service['Price']= df_metro_welder_service['PRODUCT LINK'].apply(get_metweld_price)
df_metro_welder_service['Price_Bundle']= df_metro_welder_service['BUNDLE LINK'].apply(get_metweld_price)


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/514020751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metro_welder_service['Price']= df_metro_welder_service['PRODUCT LINK'].apply(get_metweld_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/514020751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metro_welder_service['Price_Bundle']= df_metro_welder_service['BUNDLE LINK'].apply(get_metweld_price)


In [23]:
df_toolkit_depot = df_sub[df_sub['Shop Name'].str.contains('TKD', case=False, na=False, regex=True)]
df_toolkit_depot['Price']= df_toolkit_depot['PRODUCT LINK'].apply(get_toolkitdepot_price)
df_toolkit_depot['Price_Bundle']= df_toolkit_depot['BUNDLE LINK'].apply(get_toolkitdepot_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/3499184756.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toolkit_depot['Price']= df_toolkit_depot['PRODUCT LINK'].apply(get_toolkitdepot_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/3499184756.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toolkit_depot['Price_Bundle']= df_toolkit_depot['BUNDLE LINK'].apply(get_toolkitdepot_price)


In [24]:
df_supercheapauto = df_sub[df_sub['Shop Name'].str.contains('SUPERCHEAP AUTO', case=False, na=False, regex=True)]
df_supercheapauto['Price']= df_supercheapauto['PRODUCT LINK'].apply(get_supercheapauto_price)
df_supercheapauto['Price_Bundle']= df_supercheapauto['BUNDLE LINK'].apply(get_supercheapauto_price)


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/208277489.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_supercheapauto['Price']= df_supercheapauto['PRODUCT LINK'].apply(get_supercheapauto_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/208277489.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_supercheapauto['Price_Bundle']= df_supercheapauto['BUNDLE LINK'].apply(get_supercheapauto_price)


In [25]:
df_toolswarehouse = df_sub[df_sub['Shop Name'].str.contains('TOOLS WAREHOUSE', case=False, na=False, regex=True)]
df_toolswarehouse['Price']= df_toolswarehouse['PRODUCT LINK'].apply(get_toolswarehouse_price)
df_toolswarehouse['Price_Bundle']= df_toolswarehouse['BUNDLE LINK'].apply(get_toolswarehouse_price)


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1155636449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toolswarehouse['Price']= df_toolswarehouse['PRODUCT LINK'].apply(get_toolswarehouse_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1155636449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_toolswarehouse['Price_Bundle']= df_toolswarehouse['BUNDLE LINK'].apply(get_toolswarehouse_price)


In [26]:
df_vektools = df_sub[df_sub['Shop Name'].str.contains('VEK TOOLS', case=False, na=False, regex=True)]
df_vektools['Price'] = df_vektools['PRODUCT LINK'].apply(get_vektools_price)
df_vektools['Price_Bundle'] = df_vektools['BUNDLE LINK'].apply(get_vektools_price)

# Filter rows where 'PRODUCT LINK' is not NaN and not empty (after stripping spaces)
filtered_df = df_vektools[
    df_vektools['PRODUCT LINK'].notna() & (df_vektools['PRODUCT LINK'].str.strip() != "")
]

print(filtered_df[['PRODUCT LINK', 'Price', 'Price_Bundle']])



                                          PRODUCT LINK  \
148  https://www.vektools.com.au/unimig-razor-3-in-...   
435  https://www.vektools.com.au/unimig-razor-200-a...   
494  https://www.vektools.com.au/unimig-jusic-razor...   
589  https://www.vektools.com.au/unimig-kumjrr180ca...   
744  https://www.vektools.com.au/unimig-viper-cut-4...   

                                    Price  Price_Bundle  
148  <span class="price">$1,299.00</span>           NaN  
435                              1,999.00           NaN  
494                              3,480.00           NaN  
589                                445.00           NaN  
744                              1,929.00           NaN  


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/2596470196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vektools['Price'] = df_vektools['PRODUCT LINK'].apply(get_vektools_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/2596470196.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vektools['Price_Bundle'] = df_vektools['BUNDLE LINK'].apply(get_vektools_price)


In [27]:
df_kennedys = df_sub[df_sub['Shop Name'].str.contains("KENNEDY'S WELDING SUPPLIES", case=False, na=False, regex=True)]
df_kennedys['Price']= df_kennedys['PRODUCT LINK'].apply(get_kennedys_price)
df_kennedys['Price_Bundle']= df_kennedys['BUNDLE LINK'].apply(get_kennedys_price)


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/3163767015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kennedys['Price']= df_kennedys['PRODUCT LINK'].apply(get_kennedys_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/3163767015.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kennedys['Price_Bundle']= df_kennedys['BUNDLE LINK'].apply(get_kennedys_price)


In [28]:
from total_tools_scraper import get_total_tools_price
df_totaltools = df_sub[df_sub['Shop Name'].str.contains("total tools", case=False, na=False, regex=True)]
df_totaltools['Price']= df_totaltools['PRODUCT LINK'].apply(get_total_tools_price)
df_totaltools['Price_Bundle']= df_totaltools['BUNDLE LINK'].apply(get_total_tools_price)


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/2930075219.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_totaltools['Price']= df_totaltools['PRODUCT LINK'].apply(get_total_tools_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/2930075219.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_totaltools['Price_Bundle']= df_totaltools['BUNDLE LINK'].apply(get_total_tools_price)


In [29]:

def init_driver() -> webdriver.Chrome:
    """
    Initialize a headless Selenium Chrome driver and return it.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(15)
    return driver

def close_driver(driver: webdriver.Chrome):
    driver.quit()

def fetch_value_by_xpath(driver: webdriver.Chrome, url: str, xpath: str) -> str:
    """
    Given a driver, a URL, and an XPath, fetch the URL, wait for dynamic content,
    and return the text content of the element specified by the XPath.
    Returns np.nan in case of errors.
    """
    if not url or not isinstance(url, str) or url.strip() == "":
        return np.nan
    try:
        driver.get(url)
        # Adjust the sleep time as needed (or use WebDriverWait for better control)
        time.sleep(3)
        element = driver.find_element(By.XPATH, xpath)
        value = element.text.strip()
        return value.replace("A$","")
    except Exception as e:
        print("Exception occurred while fetching value for URL:", url)
        return np.nan

In [30]:
driver = init_driver()

In [31]:
# Define your XPath (make sure this is the correct path)
waindustrial_xpath_price_ = (
    "/html/body/div[1]/div/div[1]/div[1]/div/div/div[2]/div[2]/div/div[1]/div/"
    "div/div/div/div/div[2]/div/div/div/div[2]/form/section[1]/div[1]/div/h3/span"
)

# Filter your dataframe for WA INDUSTRIAL SUPPLIES WEBSITE
df_wa_industrial_website = df_sub[
    df_sub['Shop Name'].str.contains("WA INDUSTRIAL SUPPLIES WEBSITE", case=False, na=False, regex=True)
]

# Apply your scraper function on the PRODUCT LINK column
df_wa_industrial_website['Price'] = df_wa_industrial_website['PRODUCT LINK'].apply(
    lambda url: fetch_value_by_xpath(driver, url, waindustrial_xpath_price_)
)

# Similarly, apply on the BUNDLE LINK column
df_wa_industrial_website['Price_Bundle'] = df_wa_industrial_website['BUNDLE LINK'].apply(
    lambda url: fetch_value_by_xpath(driver, url, waindustrial_xpath_price_)
)


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1426356570.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wa_industrial_website['Price'] = df_wa_industrial_website['PRODUCT LINK'].apply(
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_89758/1426356570.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wa_industrial_website['Price_Bundle'] = df_wa_industrial_website['BUNDLE LINK'].apply(


In [32]:
# Define your XPath for TOOLS WAREHOUSE product pages

toolswarehouse_xpath_price = "/html/body/main/div[1]/section/article/div[2]/div/div[6]/div[1]/div/div[4]/span[2]"

# Filter your DataFrame for rows where the Shop Name contains "TOOLS WAREHOUSE"
df_toolswarehouse = df_sub[
    df_sub['Shop Name'].str.contains("TOOLS WAREHOUSE", case=False, na=False, regex=True)
].copy()

# Apply your scraper function on the PRODUCT LINK column
df_toolswarehouse['Price'] = df_toolswarehouse['PRODUCT LINK'].apply(
    lambda url: fetch_value_by_xpath(driver, url, toolswarehouse_xpath_price)
)

# Similarly, apply the scraper function on the BUNDLE LINK column
df_toolswarehouse['Price_Bundle'] = df_toolswarehouse['BUNDLE LINK'].apply(
    lambda url: fetch_value_by_xpath(driver, url, toolswarehouse_xpath_price)
)

Exception occurred while fetching value for URL: https://toolswarehouse.com.au/products/unimig-kumjr350swf-350a-razorweld-350-mig-tig-mma-dc-inverter-welder?srsltid=AfmBOor-c6ieyd7BttRHUs2dHmnlAabaQRkpaaaoTGB5BJUDNYI7T2x8


In [33]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver_sydney = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
sydney_tools_xpath_price="/html/body/div[1]/div/div/section/section/div[2]/div/div/div[3]/div[2]/div[3]/div/div[2]/span[2]"
df_sydney_tools = df_sub[df_sub['Shop Name'].str.contains("SYDNEY TOOLS", case=False, na=False, regex=True)].copy()
df_sydney_tools['Price'] = df_sydney_tools['PRODUCT LINK'].apply(
    lambda url: get_sydney_tools_price(url,driver)
)
# Similarly, apply the scraper function on the BUNDLE LINK column
df_sydney_tools['Price_Bundle'] = df_sydney_tools['BUNDLE LINK'].apply(
    lambda url: get_sydney_tools_price(url,driver)
)
# Now print or further process your DataFrame
driver_sydney.quit()

In [34]:
def get_hares_and_forbes_price(url, driver):
    try:
        # Check if the URL is valid and starts with the expected domain.
        if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
            return np.nan
        if not url.startswith("https://www.machineryhouse.com.au"):
            return np.nan

        xpath_price = "/html/body/div[1]/div[3]/main/section/div/div[4]/div[2]/div[1]/div[3]/div/div[2]/span"
        driver.get(url)
        time.sleep(5)  # Adjust as necessary based on your connection and the site's response
        
        price_element = driver.find_element(By.XPATH, xpath_price)
        return price_element.text
    except Exception as e:
        # Log the error if needed (print or use logging)
        print(f"Error retrieving price from {url}: {e}")
        return np.nan

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-dev-shm-usage")

driver_hare = webdriver.Chrome(options=chrome_options)

df_hare_and_forbes = df_sub[df_sub['Shop Name'].str.contains("hare and forbes", case=False, na=False, regex=True)].copy()
df_hare_and_forbes['Price'] = df_hare_and_forbes['PRODUCT LINK'].apply(
    lambda url: get_hares_and_forbes_price(url,driver_hare)
)

# Similarly, apply the scraper function on the BUNDLE LINK column
df_hare_and_forbes['Price_Bundle'] = df_hare_and_forbes['BUNDLE LINK'].apply(
    lambda url: get_hares_and_forbes_price(url,driver_hare)
)

driver_hare.quit()


Error retrieving price from https://www.machineryhouse.com.au/w194: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/main/section/div/div[4]/div[2]/div[1]/div[3]/div/div[2]/span"}
  (Session info: chrome=134.0.6998.118); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001006276c8 cxxbridge1$str$ptr + 2791212
1   chromedriver                        0x000000010061fc9c cxxbridge1$str$ptr + 2759936
2   chromedriver                        0x0000000100171e30 cxxbridge1$string$len + 92928
3   chromedriver                        0x00000001001b9170 cxxbridge1$string$len + 384576
4   chromedriver                        0x00000001001fa5f8 cxxbridge1$string$len + 651976
5   chromedriver                        0x00000001001ad2fc cxxbridge1$string$len + 335820
6   chromedriver             

In [35]:
df_hare_and_forbes


,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment,Price,Price_Bundle
32,Unimig,U11006K,VIPER 165,HARE AND FORBES,https://www.machineryhouse.com.au/w2442,NaN,NaN,NaN,649.00,NaN
44,Unimig,KUMJRVM185,VIPER 185,HARE AND FORBES,https://www.machineryhouse.com.au/w243,NaN,NaN,NaN,879.00,NaN
57,Unimig,KUMJRVM185,VIPER 185,HARE AND FORBES,https://www.machineryhouse.com.au/w243,NaN,NaN,NaN,879.00,NaN
69,Unimig,U11011,VIPER 195,HARE AND FORBES,https://www.machineryhouse.com.au/w2431,NaN,NaN,NaN,"1,122.00",NaN
80,Unimig,U11011,VIPER 195,HARE AND FORBES,https://www.machineryhouse.com.au/w2431,NaN,NaN,NaN,"1,122.00",NaN
92,Unimig,U11003K,RAZOR 200 PULSE,HARE AND FORBES,https://www.machineryhouse.com.au/w194,NaN,NaN,NaN,NaN,NaN
112,Unimig,U11004K,RAZOR 230,HARE AND FORBES,https://www.machineryhouse.com.au/w168,NaN,NaN,NaN,"2,266.00",NaN
131,Unimig,U11007K,RAZOR MULTI 175,HARE AND FORBES,https://www.machineryhouse.com.au/w1771,NaN,NaN,NaN,"1,166.00",NaN
153,Unimig,U11012,RAZOR MULTI 200 PFC,HARE AND FORBES,NaN,NaN,NaN,NaN,NaN,NaN
178,Unimig,U11008K,RAZOR MULTI 220,HARE AND FORBES,https://www.machineryhouse.com.au/w1786,NaN,NaN,NaN,"1,485.00",NaN


In [36]:
driver.close()

In [15]:
from alphaweld import get_alphaweld_price

# Filter the DataFrame for Alphaweld shop entries
df_alphaweld = df_sub[df_sub['Shop Name'].str.contains("alphaweld", case=False, na=False, regex=True)]

# Apply the price extraction function
df_alphaweld['Price'] = df_alphaweld['PRODUCT LINK'].apply(get_alphaweld_price)
df_alphaweld['Price_Bundle'] = df_alphaweld['BUNDLE LINK'].apply(get_alphaweld_price)

# Count non-NaN values in the PRICE column
non_nan_count = df_alphaweld['Price'].count()
total_count = len(df_alphaweld)
print(f"Non-NaN values in PRICE column: {non_nan_count} out of {total_count}")

# Print the PRODUCT LINK URLs that returned NaN for Price
nan_urls = df_alphaweld[df_alphaweld['Price'].isna()]['PRODUCT LINK']
if not nan_urls.empty:
    print("The following PRODUCT LINK URLs returned NaN for Price:")
    for url in nan_urls:
        print(url)

Non-NaN values in PRICE column: 16 out of 36
The following PRODUCT LINK URLs returned NaN for Price:
nan
https://www.alphaweld.com.au/product/9214-unimig-razor-350-swf-mig-tig-stick-welder-package
https://www.alphaweld.com.au/product/21214-unimig-razor-500-swf-mig-tig-stick-welder
https://www.alphaweld.com.au/product/19981-unimig-razor-320-ac-dc-tig-welder?srsltid=AfmBOorp7Dkp2423b8U3rW4cLdQvsWmiL2_yer_f3JQr1spi8TREOmgy
nan
nan
nan
nan
nan
nan
https://www.alphaweld.com.au/product/19435-razor-cut-80-plasma-cutter-machine-415v?srsltid=AfmBOopwqvdvNWVKfeitRGBnNnkEgqj-bJczGrf-a-FxNrOU-6KtgwlB
nan
nan
nan
nan
nan
nan
nan
nan
nan


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_93025/2782441997.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alphaweld['Price'] = df_alphaweld['PRODUCT LINK'].apply(get_alphaweld_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_93025/2782441997.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alphaweld['Price_Bundle'] = df_alphaweld['BUNDLE LINK'].apply(get_alphaweld_price)


In [17]:
import httpx
import numpy as np
import pandas as pd
from parsel import Selector

def get_hampdon_price(url: str) -> str:
    
    # Validate the URL
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if not url.startswith("https://www.hampdon.com.au/"):
        return np.nan

    try:
        with httpx.Client(
            headers={
                "User-Agent": (
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/113.0.0.0 Safari/537.36"
                ),
                "Accept": (
                    "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,"
                    "image/webp,*/*;q=0.8"
                ),
            },
            follow_redirects=True,
            timeout=15.0,
        ) as client:
            response = client.get(url)
    except (httpx.RequestError, httpx.ReadTimeout, Exception):
        return np.nan

    # Parse the response text with Selector
    sel = Selector(response.text)
    
    # Locate the price element using the appropriate CSS selector
    price_element = sel.css("div.productprice.productpricetext[itemprop='price']")
    if not price_element:
        return np.nan

    # First, try to extract the price using the "content" attribute
    content_attr = price_element.attrib.get("content", "")
    if content_attr:
        cleaned_price = content_attr.strip()
        return cleaned_price

    # Fallback: extract the inner text and clean it
    price_text = price_element.css("::text").get()
    if not price_text:
        return np.nan
    cleaned_price = price_text.replace("$", "").replace(",", "").strip()
    
    return cleaned_price

if __name__ == "__main__":
    url = "https://www.hampdon.com.au/unimig-razor-tig-200-ac/dc-water-cooler?srsltid=AfmBOooUly0f8iVOdfNeit9ErrVNrUeWi8OdnXLWZ4ZwcJqbb9mrO40C"
    price = get_hampdon_price(url)
    print("Final extracted price:", price)


Final extracted price: 1445


In [19]:
# Filter the DataFrame for Alphaweld shop entries
df_hampdon = df_sub[df_sub['Shop Name'].str.contains("hampdon website", case=False, na=False, regex=True)]

# Apply the price extraction function
df_hampdon['Price'] = df_hampdon['PRODUCT LINK'].apply(get_hampdon_price)
df_hampdon['Price_Bundle'] = df_hampdon['BUNDLE LINK'].apply(get_hampdon_price)

# Count non-NaN values in the PRICE column
non_nan_count = df_hampdon['Price'].count()
total_count = len(df_hampdon)
print(f"Non-NaN values in PRICE column: {non_nan_count} out of {total_count}")

# Print the PRODUCT LINK URLs that returned NaN for Price
nan_urls = df_hampdon[df_hampdon['Price'].isna()]['PRODUCT LINK']
if not nan_urls.empty:
    print("The following PRODUCT LINK URLs returned NaN for Price:")
    for url in nan_urls:
        print(url)

Non-NaN values in PRICE column: 1 out of 31
The following PRODUCT LINK URLs returned NaN for Price:
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_93025/3031084728.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hampdon['Price'] = df_hampdon['PRODUCT LINK'].apply(get_hampdon_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_93025/3031084728.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hampdon['Price_Bundle'] = df_hampdon['BUNDLE LINK'].apply(get_hampdon_price)


In [29]:
import httpx
import numpy as np
import pandas as pd
from parsel import Selector

def get_national_welding_price(url: str) -> str:
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if not url.startswith("https://www.nationalwelding.com.au/"):
        return np.nan
    try:
        with httpx.Client(
            headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8"
            },
            follow_redirects=True,
            timeout=15.0,
        ) as client:
            response = client.get(url)
    except (httpx.RequestError, httpx.ReadTimeout, Exception):
        return np.nan
    sel = Selector(response.text)
    promo_price_element = sel.css("div.productpromo[itemprop='price']")
    if promo_price_element:
        content_attr = promo_price_element.attrib.get("content", "")
        if content_attr:
            return content_attr.strip()
        promo_price_text = promo_price_element.css("::text").get()
        if promo_price_text:
            return promo_price_text.replace("NOW", "").replace("$", "").replace(",", "").strip()
    price_element = sel.css("div.productprice.productpricetext[itemprop='price']")
    if not price_element:
        return np.nan
    content_attr = price_element.attrib.get("content", "")
    if content_attr:
        return content_attr.strip()
    price_text = price_element.css("::text").get()
    if not price_text:
        return np.nan
    return price_text.replace("$", "").replace(",", "").strip()

if __name__ == "__main__":
    url = "https://www.nationalwelding.com.au/unimig-viper-multi-195-max-welder-u11011?srsltid=AfmBOorwGPQVjdFtBS0Lq0iL7htZ-DDhidL9QyhL-g0rKZeX8E4gN9Vm"
    price = get_national_welding_price(url)
    print("Final extracted price:", price)


Final extracted price: 1199


In [35]:
def get_price(url: str) -> str:
    if not isinstance(url, str):
        return np.nan
    if url.startswith("https://www.ebay.com.au/"):
        return get_ebay_price(url)
    return get_national_welding_price(url)

df_national_welding = df_sub[df_sub['Shop Name'].str.contains("national welding", case=False, na=False, regex=True)]
df_national_welding['Price'] = df_national_welding['PRODUCT LINK'].apply(get_price)
df_national_welding['Price_Bundle'] = df_national_welding['BUNDLE LINK'].apply(get_price)

non_nan_count = df_national_welding['Price'].count()
total_count = len(df_national_welding)
print(f"Non-NaN values in PRICE column: {non_nan_count} out of {total_count}")

nan_urls = df_national_welding[df_national_welding['Price'].isna()]['PRODUCT LINK']
if not nan_urls.empty:
    print("The following PRODUCT LINK URLs returned NaN for Price:")
    for url in nan_urls:
        print(url)


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_93025/479573420.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_national_welding['Price'] = df_national_welding['PRODUCT LINK'].apply(get_price)


Non-NaN values in PRICE column: 42 out of 72
The following PRODUCT LINK URLs returned NaN for Price:
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_93025/479573420.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_national_welding['Price_Bundle'] = df_national_welding['BUNDLE LINK'].apply(get_price)


In [ ]:
df_national_welding = df_sub[df_sub['Shop Name'].str.contains("", case=False, na=False, regex=True)]
df_national_welding['Price'] = df_national_welding['PRODUCT LINK'].apply(get_price)
df_national_welding['Price_Bundle'] = df_national_welding['BUNDLE LINK'].apply(get_price)

non_nan_count = df_national_welding['Price'].count()
total_count = len(df_national_welding)
print(f"Non-NaN values in PRICE column: {non_nan_count} out of {total_count}")

nan_urls = df_national_welding[df_national_welding['Price'].isna()]['PRODUCT LINK']
if not nan_urls.empty:
    print("The following PRODUCT LINK URLs returned NaN for Price:")
    for url in nan_urls:
        print(url)

In [37]:
combined_df = pd.concat([df_national_welding,df_hampdon,df_alphaweld,df_sydney_tools,df_vektools,df_totaltools,df_wa_industrial_website,df_kennedys,df_toolswarehouse,df_ebay, df_electroweld_website, df_supercheapauto,df_biba_website,df_gentronics,df_weld_com,df_weldconnect,df_metro_welder_service,df_toolkit_depot], ignore_index=True)

# Sort the combined dataframe by the "PRODUCT NAME" column
combined_df.sort_values("PRODUCT NAME", inplace=True)

NameError: name 'df_sydney_tools' is not defined

In [ ]:
combined_df.info()

In [ ]:
combined_df.to_csv('combined.csv',index=False)

In [ ]:
completed_unique_shops = combined_df['Shop Name'].unique()
print(completed_unique_shops)
for i in completed_unique_shops:
    print(i)

In [ ]:
unique_shops = df_sub['Shop Name'].unique()

In [ ]:
not_completed = list(set(unique_shops) - set(completed_unique_shops))
for i in not_completed:
    print(i)
print(str(len(not_completed)-1)+"/"+str((len(unique_shops)-1)))

In [ ]:
# Assuming these data frames are defined:
# df_kennedys, df_toolswarehouse, df_ebay, df_electroweld_website, 
# df_supercheapauto, df_biba_website, df_gentronics, df_weld_com, 
# df_weldconnect, df_metro_welder_service, df_toolkit_depot

dfs = {
    "df_kennedys": df_kennedys,
    "df_toolswarehouse": df_toolswarehouse,
    "df_ebay": df_ebay,
    "df_electroweld_website": df_electroweld_website,
    "df_supercheapauto": df_supercheapauto,
    "df_biba_website": df_biba_website,
    "df_gentronics": df_gentronics,
    "df_weld_com": df_weld_com,
    "df_weldconnect": df_weldconnect,
    "df_metro_welder_service": df_metro_welder_service,
    "df_toolkit_depot": df_toolkit_depot,
}

for name, df in dfs.items():
    print(f"{name} has {len(df)} rows")


In [ ]:
    # Close the driver when done.
    close_driver(driver)